<a href="https://colab.research.google.com/github/Kshitij879/PnemoniaDetection/blob/main/Pneumonia_Detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>